In [3]:
import os
os.chdir('/Users/admin/Downloads/Deep Learning Project')

In [16]:
'''
네이버 경제뉴스 홈페이지 스크롤링 v4.0!!!
저자: 육태경, 2023/04/22
이 코드는 현제 네이버뉴스, 경제 종목포탈 홈페이지안에 존재하는 페이지링크들을 코드에 세팅된 페이지 수 많큼 'econ_homepage_links.csv'에
같은 디렉토리안에 csv파일로 저장하는 스크롤링 프로그램입니다. *주의 이 프로그램은 굉장히 원시적인 프로그램으로, base_url의 값이 네이버 외의 포탈
주소를 값으로 받아들이면 에러가 날 수 있습니다.
네이버뉴스안에서만 작동하며, 광범위하게 쓰려면 다른 사이트의 url주소 특징을 파악해
다른 알고리즘을 적용해야합니다.
'''
import datetime
import sys

from requests_html import HTMLSession
import csv

session = HTMLSession()

def get_links(url, start_day, end_day):
    r = session.get(url)
    links = r.html.absolute_links
    filtered_links = [
        link for link in links
        if 'https://n.news.naver.com/mnews/article' in link
        and get_date(link) is not None
        and start_day <= get_date(link) <= end_day
    ]
    return filtered_links


def get_date(url):
    r = session.get(url)
    date_element = r.html.find('.t11', first=True)
    
    if date_element is None:
        return None

    date_str = date_element.text
    date_obj = datetime.datetime.strptime(date_str, '%Y.%m.%d. %H:%M')
    return date_obj.date()


def main(start_day, end_day):
    base_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page='
    num_pages = 3
    all_links = []
    
    start_day = datetime.datetime.strptime(start_day, '%Y%m%d').date()
    end_day = datetime.datetime.strptime(end_day, '%Y%m%d').date()

    for page in range(1, num_pages + 1):
        url = base_url + str(page)
        links = get_links(url, start_day, end_day)
        all_links.extend(links)

    with open('econ_hompage_links.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])

if __name__ == '__main__':
    start_day = '20230420'
    end_day = '20230422'
    main(start_day, end_day)

In [8]:
'''
네이버 경제뉴스 홈페이지 스크롤링 v3.0!!!
저자: 육태경, 2023/04/22
이 코드는 현제 네이버뉴스, 경제 종목포탈 홈페이지안에 존재하는 페이지링크들을 코드에 세팅된 페이지 수 많큼 'econ_homepage_links.csv'에
같은 디렉토리안에 csv파일로 저장하는 스크롤링 프로그램입니다. *주의 이 프로그램은 굉장히 원시적인 프로그램으로, base_url의 값이 네이버 외의 포탈
주소를 값으로 받아들이면 에러가 날 수 있습니다. 홈페이지는 현재시간으로부터 4일전까지만 뉴스내용 저장함. 5일이상 넘은것은 오류날 수 있음.
네이버뉴스안에서만 작동하며, 광범위하게 쓰려면 다른 사이트의 url주소 특징을 파악해
다른 알고리즘을 적용해야합니다.
'''
from requests_html import HTMLSession
import csv

# Create a session object for making HTTP requests
session = HTMLSession()

def get_links(url):
    # Make an HTTP GET request to the specified URL
    r = session.get(url)
    # Extract all the links from the HTML response
    links = r.html.absolute_links
    # Filter the links to only include news articles
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    # Return the filtered list of links
    return filtered_links

def main():
    # 베이스 홈페이지 주소 세팅
    base_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page='
    # !!!!!!!!**중요**!!!!!!! 여기서 원하는 페이지의 양 조절 가능함.
    num_pages = 30

    # all_links라는 링크 저장소 텅빈 리스트를 만든다.
    all_links = []
    # 홈페이지 주소안에 있는 페이지들에 들어있는 링크들의 주소들을 all_link로 옮겨 담는다.
    
    #num_pages에 적혀진 수 많큼 루핑을 한다
    for page in range(1, num_pages + 1):
        # 현제 url주소를 구축
        url = base_url + str(page)
        # 현제 페이지에서 링크 주소 값 links로 옮김
        links = get_links(url)
        # links값을 all_links로 옮김
        all_links.extend(links)

    # 새로운 CSV 파일을 열어 링크들을 다 저장함.
    with open('econ_hompage_links.csv', 'w', newline='', encoding='utf-8') as f:
        # 새 CSV 라이터 오브젝트를 만듬
        writer = csv.writer(f)
        # 헤더 열을 CSV 파일에다가 씀
        writer.writerow(['Links'])
        # all_link에 있는 링크들을 모두 CSV 파일에 행 데이터로 씀
        writer.writerows([[link] for link in all_links])

# 스크립트가 성공적이었다면, main함수를 작동!
if __name__ == '__main__':
    main()

In [13]:
'''
네이버 경제뉴스 종목별 스크롤링 v3.0!!!
저자: 육태경, 2023/04/24
이 코드는 현제 네이버뉴스, 경제 종목포탈 홈페이지안에 존재하는 페이지링크들을 종목별로 코드에 세팅된 페이지 수,
기간많큼 'econ_homepage_links.csv'에 같은 디렉토리안에 csv파일로 저장하는 스크롤링 프로그램입니다.
*주의 이 프로그램은 굉장히 원시적인 프로그램으로, base_url의 값이 네이버 외의 포탈
주소를 값으로 받아들이면 에러가 날 수 있습니다. 네이버뉴스안에서만 작동하며, 광범위하게 쓰려면 다른 사이트의 url주소 특징을 파악해
다른 알고리즘을 적용해야합니다.*
'''
import datetime

from requests_html import HTMLSession
import csv

# HTTP 요청을 위한 세션 오브젝트 생성
session = HTMLSession()

def get_links(url):
    # 지정된 URL로 HTTP GET 요청을 보냄
    r = session.get(url)
    # HTML 응답에서 모든 링크를 추출
    links = r.html.absolute_links
    # 링크를 필터링하여 뉴스 기사만 포함
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    # 필터링된 링크 목록 반환
    return filtered_links

def main(start_day, end_day):
    # Convert the input dates into datetime objects
    start_day = datetime.datetime.strptime(start_day, '%Y%m%d')
    end_day = datetime.datetime.strptime(end_day, '%Y%m%d')

    # Calculate the number of days between the start and end dates
    days_difference = (end_day - start_day).days

    # 각 sid2 값에 대해 스크랩 할 페이지 수 설정
    num_pages = 3

    # 모든 링크를 저장할 빈 리스트 생성
    all_links = []

    # 각 sid2 값에 대해 반복
    for sid2_value in [259, 258, 261, 771, 260, 262, 310, 263]:
        # 각 sid2 값에 대한 카테고리 이름 추가
        if sid2_value == 259:
            all_links.append('금융')
        elif sid2_value == 258:
            all_links.append('증권')
        elif sid2_value == 261:
            all_links.append('산업/재계')
        elif sid2_value == 771:
            all_links.append('중기/벤처')
        elif sid2_value == 260:
            all_links.append('부동산')
        elif sid2_value == 262:
            all_links.append('글로벌 경제')
        elif sid2_value == 310:
            all_links.append('생활경제')
        elif sid2_value == 263:
            all_links.append('경제 일반')

        # 현재 sid2 값에 대한 기본 URL 구성
        base_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}'
        # 기본 URL에서 링크 가져오기
        links = get_links(base_url)
        # all_links 목록에 링크 추가
        all_links.extend(links)

        # 각 페이지 번호에 대해 반복
        for day_delta in range(days_difference + 1):
            current_day = (start_day + datetime.timedelta(days=day_delta)).strftime('%Y%m%d')

            for page in range(1, num_pages + 1):
                # 현재 페이지 및 sid2 값에 대한 URL 구성
                url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}&date={current_day}&page={page}'
                # 현재 URL에서 링크 가져오기
                links = get_links(url)
                # all_links 목록에 링크 추가
                all_links.extend(links)
                
        # 링크를 저장하기 위해 새 CSV 파일 열기
    with open('econ_categorical_links.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])
        

# 스크립트가 성공적이었다면, main함수를 작동!
if __name__ == '__main__':
    start_day = '20230420'
    end_day = '20230422'
    main(start_day, end_day)



In [9]:
'''
네이버 경제뉴스 종목별 스크롤링 v2.0!!!
저자: 육태경, 2023/04/22
이 코드는 현제 네이버뉴스, 경제 종목포탈 홈페이지안에 존재하는 페이지링크들을 종목별로 코드에 세팅된 페이지 수 많큼 'econ_homepage_links.csv'에
같은 디렉토리안에 csv파일로 저장하는 스크롤링 프로그램입니다. *주의 이 프로그램은 굉장히 원시적인 프로그램으로, base_url의 값이 네이버 외의 포탈
주소를 값으로 받아들이면 에러가 날 수 있습니다.
네이버뉴스안에서만 작동하며, 광범위하게 쓰려면 다른 사이트의 url주소 특징을 파악해
다른 알고리즘을 적용해야합니다.
'''
from requests_html import HTMLSession
import csv

# HTTP 요청을 위한 세션 오브젝트 생성
session = HTMLSession()

def get_links(url):
    # 지정된 URL로 HTTP GET 요청을 보냄
    r = session.get(url)
    # HTML 응답에서 모든 링크를 추출
    links = r.html.absolute_links
    # 링크를 필터링하여 뉴스 기사만 포함
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    # 필터링된 링크 목록 반환
    return filtered_links

# 메인 함수 정의
def main(start_day, end_day):
    # 각 sid2 값에 대해 스크랩 할 페이지 수 설정
    num_pages = 3

    # 모든 링크를 저장할 빈 리스트 생성
    all_links = []

    # 각 sid2 값에 대해 반복
    for sid2_value in [259, 258, 261, 771, 260, 262, 310, 263]:
        # 각 sid2 값에 대한 카테고리 이름 추가
        if sid2_value == 259:
            all_links.append('금융')
        elif sid2_value == 258:
            all_links.append('증권')
        elif sid2_value == 261:
            all_links.append('산업/재계')
        elif sid2_value == 771:
            all_links.append('중기/벤처')
        elif sid2_value == 260:
            all_links.append('부동산')
        elif sid2_value == 262:
            all_links.append('글로벌 경제')
        elif sid2_value == 310:
            all_links.append('생활경제')
        elif sid2_value == 263:
            all_links.append('경제 일반')

            # 현재 sid2 값에 대한 기본 URL 구성
        base_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}'
        # 기본 URL에서 링크 가져오기
        links = get_links(base_url)
        # all_links 목록에 링크 추가
        all_links.extend(links)

        # 각 페이지 번호에 대해 반복
        for page in range(2, num_pages + 1):
            # 현재 페이지 및 sid2 값에 대한 URL 구성
            url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}&date=20230422&page={page}'
            # 현재 URL에서 링크 가져오기
            links = get_links(url)
            # all_links 목록에 링크 추가
            all_links.extend(links)

    # 링크를 저장하기 위해 새 CSV 파일 열기
    with open('econ_categorical_links.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])
        
# 스크립트가 성공적이었다면, main함수를 작동!
if __name__ == '__main__':
    main()

In [33]:
'''from requests_html import HTMLSession
import csv

session = HTMLSession()

def get_links(url):
    r = session.get(url)
    links = r.html.find('.list_news a')
    return links

def main():
    base_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page={}'
    num_pages = 40  # number of pages to scrape
    links = []
    for page in range(1, num_pages+1):
        url = base_url + str(page)
        links += get_links(url)
    links = set(links)  # remove duplicates
    with open('links.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link.attrs['href']] for link in links])

if __name__ == '__main__':
    main()
'''

In [40]:
'''#경제 홈페이지 안에 있는 종목별 스크롤링 v1.0
from requests_html import HTMLSession
import csv

session = HTMLSession()

def get_links(url): 
    r = session.get(url)
    links = r.html.absolute_links
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    return filtered_links

def main():
    num_pages = 3  # You can set this to the number of pages you want to scrape for each sid2 value
    all_links = []

    for sid2_value in range(259, 264):
        base_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}'
        links = get_links(base_url)
        all_links.extend(links)

        for page in range(2, num_pages + 1):
            url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}&date=20230422&page={page}'
            links = get_links(url)
            all_links.extend(links)

    with open('econ_categorical_links.csv', 'w', newline='', encoding='cp949') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])

if __name__ == '__main__':
    main()'''

In [41]:
'''#경제 홈페이지 스크롤링 v3.0
def main(): 
    base_url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page='
    num_pages = 3  # You can set this to the number of pages you want to scrape

    all_links = []
    for page in range(1, num_pages + 1):
        url = base_url + str(page)
        links = get_links(url)
        all_links.extend(links)

    with open('econ_hompage_links.csv', 'w', newline='', encoding='cp949') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])

if __name__ == '__main__':
    main()'''

In [36]:
'''from requests_html import HTMLSession
import csv

session = HTMLSession()

def get_links(url):
    r = session.get(url)
    links = r.html.absolute_links
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    return filtered_links

def main():
    url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page=1'
    links = get_links(url)
    with open('links.csv', 'w', newline='', encoding='cp949') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in links])

if __name__ == '__main__':
    main()'''

In [28]:
'''from requests_html import HTMLSession
import csv

session = HTMLSession()

def get_links(url):
    r = session.get(url)
    links = r.html.absolute_links
    return links

def main():
    url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101'
    links = get_links(url)
    with open('links.csv', 'w', newline='', encoding='cp949') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in links])

if __name__ == '__main__':
    main()
'''

In [ ]:
'''keywords = ['주식']'''

In [ ]:
'''from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# ... (Assuming you've already set up the webdriver as `driver`)

time.sleep(1)

for keyword in keywords:
    #1. Click the search bar, clear it, and input the keyword
    search_bar = driver.find_element(By.CSS_SELECTOR, '.search-key')
    search_bar.click()
    search_bar.clear()
    search_bar.send_keys(keyword)

    #2. Scroll and click the search result
    while True:
        try:
            search_btn = driver.find_element(By.CSS_SELECTOR, '.keyword-item')
            search_btn.click()
            break
        except:
            driver.execute_script("window.scrollBy(0, -300)")
    
    #3. Apply search conditions
    while True:
        try:
            step3_tab = driver.find_element(By.CSS_SELECTOR, '.btn.btn-search.news-search-btn.news-report-search-btn')
            step3_tab.click()
            break
        except:
            driver.execute_script("window.scrollBy(0, -300)")
    
    #4. Scroll and click Step 3: Analyze results and visualization
    while True:
        try:
            excel_btn = driver.find_element(By.CSS_SELECTOR, '.btn-collap-group.btn-collap')
            excel_btn.click()
            break
        except:
            driver.execute_script("window.scrollBy(0, -300)")
    
    #5. Click the download button
    while True:
        try:
            close_btn = driver.find_element(By.CSS_SELECTOR, '.btn.btn-wh.ml-auto.news-download-btn.mobile-excel-download')
            close_btn.click()
            break
        except:
            time.sleep(1)'''

In [ ]:
'''time.sleep(1)

for keyword in keywords:
    #1. 검색어 창을 클릭해서 검색어를 입력
    search_bar = driver.find_element_by_class_name('search-key')
    search_bar.click()
    search_bar.clear()
    search_bar.send_keys(keyword)

    #2. 스크롤을 내려서 검색 결과 클릭
    while True:
        try:
            search_btn = driver.find_element_by_class_name('keyword-item')
            search_btn.click()
            break
        except:
            scroll(-300)
    #3. 검색 조건 적용
    while True:
        try:
            step3_tab = driver.find_element_by_class_name('btn btn-search news-search-btn news-report-search-btn')
            step3_tab.click()
            break
        except:
            scroll(-300)
    #4. 스크롤해서 스텝 3:분석 결과 및 시각화 클릭
    while True:
        try:
            excel_btn = driver.find_element_by_class_name('btn-collap-group btn-collap')
            excel_btn.click()
            break
        except:
            scroll(-300)
    #5. 다운로드 버튼 클릭
    while True:
        try:
            close_btn = driver.find_element_by_class_name('btn btn-wh ml-auto news-download-btn mobile-excel-download')
            close_btn.click()
            break
        except:
            time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0)")
    time.sleep(1)
    step1_tab = driver.find_element_by_class_name('tap_step1')
    step1_tab.click()'''


In [9]:
'''def scroll(line):
    '''
    '''아래로 스크롤 : -
    위로 스크롤 : +'''
    '''
    time.sleep(0.5)
    pyautogui.scroll(line)
    time.sleep(0.5)'''

In [15]:
'''import csv
from requests_html import HTMLSession

# Replace the example URL with the specific website you want to scrape
url = "https://www.bigkinds.or.kr/v2/news/search.do"

session = HTMLSession()
response = session.get(url)

# Replace '.news-link' with the appropriate CSS selector for the links you want to extract
news_links = response.html.find('.news-detail')

# Extract the URLs and store them in a list
urls = [link.attrs['href'] for link in news_links]

# Save the URLs to a CSV file
with open('news_links.csv', 'w', newline='', encoding='cp949') as csvfile:
    writer = csv.writer(csvfile)
    for url in urls:
        writer.writerow([url])'''


In [45]:
'''
네이버 경제뉴스 종목별 스크롤링 v2.0!!!
저자: 육태경, 2023/04/22
이 코드는 현제 네이버뉴스, 경제 종목포탈 홈페이지안에 존재하는 페이지링크들을 종목별로 코드에 세팅된 페이지 수 많큼 'econ_homepage_links.csv'에
같은 디렉토리안에 csv파일로 저장하는 스크롤링 프로그램입니다. *주의 이 프로그램은 굉장히 원시적인 프로그램으로, base_url의 값이 네이버 외의 포탈
주소를 값으로 받아들이면 에러가 날 수 있습니다.
네이버뉴스안에서만 작동하며, 광범위하게 쓰려면 다른 사이트의 url주소 특징을 파악해
다른 알고리즘을 적용해야합니다.

from requests_html import HTMLSession
import csv

# Create a session object for making HTTP requests
session = HTMLSession()

def get_links(url):
    # Make an HTTP GET request to the specified URL
    r = session.get(url)
    # Extract all the links from the HTML response
    links = r.html.absolute_links
    # Filter the links to only include news articles
    filtered_links = [link for link in links if 'https://n.news.naver.com/mnews/article' in link]
    # Return the filtered list of links
    return filtered_links

# Define the main function
def main():
    # Set the number of pages to scrape for each sid2 value
    num_pages = 3

    # Create an empty list to store all the links
    all_links = []
 
    # Loop through each sid2 value
    for sid2_value in [259, 258, 261, 771, 260, 262, 310, 263]:
        # Add category names for each sid2 value
        if sid2_value == 259:
            all_links.append('금융')
        elif sid2_value == 258:
            all_links.append('증권')
        elif sid2_value == 261:
            all_links.append('산업/재계')
        elif sid2_value == 771:
            all_links.append('중기/벤처')
        elif sid2_value == 260:
            all_links.append('부동산')
        elif sid2_value == 262:
            all_links.append('글로벌 경제')
        elif sid2_value == 310:
            all_links.append('생활경제')
        elif sid2_value == 263:
            all_links.append('경제 일반')



        # Construct the base URL for the current sid2 value
        base_url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}'
        # Get the links from the base URL
        links = get_links(base_url)
        # Add the links to the all_links list
        all_links.extend(links)

        # Loop through each page number
        for page in range(2, num_pages + 1):
            # Construct the URL for the current page and sid2 value
            url = f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2={sid2_value}&date=20230422&page={page}'
            # Get the links from the current URL
            links = get_links(url)
            # Add the links to the all_links list
            all_links.extend(links)

    # Open a new CSV file to save the links
    with open('econ_categorical_links.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Links'])
        writer.writerows([[link] for link in all_links])
        
# 스크립트가 성공적이었다면, main함수를 작동!
if __name__ == '__main__':
    main()'''